In [1]:
import json, csv
import numpy as np
import pandas as pd

# df = pd.read_csv('../datasets/entities_nlp_train_hongyu.csv')
df = pd.read_csv('../datasets/all-nlp-features.csv')
df[20:30:2]

,id,keyword,location,text,entities,labels,ent_dep,ent_head,ent_pos,ent_children,...,contains_l1_synonyms,contains_l2_synonyms,contains_damaged_words,mentioned_news_org,mentioned_relief_org,mentions,orgs,gpes,facs,target
20,31,missing,unknown,this is ridiculous....,None,None,None,None,None,None,...,False,False,False,False,False,[],[],[],[],0.0
22,33,missing,unknown,Love skiing,None,None,None,None,None,None,...,False,False,False,False,False,[],[],[],[],0.0
24,36,missing,unknown,LOOOOOOL,None,None,None,None,None,None,...,False,False,False,False,False,[],[],[],[],0.0
26,38,missing,unknown,Was in NYC last week!,"NYC,last week","ORG,DATE",pobj,in,ADP,None,...,False,False,False,False,False,[],['NYC'],[],[],0.0
28,40,missing,unknown,Cooool :),None,None,None,None,None,None,...,False,False,False,False,False,[],[],[],[],0.0


## evaluation

In [2]:
y = df['target']
y.shape

(7503,)

In [3]:
X = df.drop(['target'], axis=1)
X.columns

Index(['id', 'keyword', 'location', 'text', 'entities', 'labels', 'ent_dep',
       'ent_head', 'ent_pos', 'ent_children', 'hashtag',
       'hashtags_have_l1_synonyms', 'hashtags_have_l2_synonyms', 'subj',
       'verb', 'obj', 'contains_l1_synonyms', 'contains_l2_synonyms',
       'contains_damaged_words', 'mentioned_news_org', 'mentioned_relief_org',
       'mentions', 'orgs', 'gpes', 'facs'],
      dtype='object')

In [7]:
from sklearn.pipeline import FeatureUnion, Pipeline, make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer

class LabeledNormalizer(Normalizer):
    def fit(self, X, *args, **kwargs):
        try:
            self.names = X.columns
        except:
            self.names = [str(i) for i in range(X.shape[1])]
        return super().fit(X, *args, **kwargs)
         
    def get_feature_names(self):
        return self.names
    
vec = ColumnTransformer([
#     ('norm', LabeledNormalizer(), ['id']),
    ('kw', TfidfVectorizer(ngram_range=(1, 3), min_df=2, token_pattern=r"(?u)\b\w+\b",), 'keyword'),
    ('loc', TfidfVectorizer(ngram_range=(1, 3), min_df=2, token_pattern=r"(?u)\b\w+\b"), 'location'),
    ('text', TfidfVectorizer(ngram_range=(1, 3), min_df=2, token_pattern=r"(?u)\b\w+\b"), 'text'),
    ('ent', CountVectorizer(ngram_range=(1, 1), analyzer='word', token_pattern=r"(?u)\b\w+\b"), 'entities'),
    ('label', CountVectorizer(ngram_range=(1, 1), analyzer='word', token_pattern=r"(?u)\b\w+\b"), 'labels'),
    ('dep', CountVectorizer(ngram_range=(1, 1), analyzer='word', token_pattern=r"(?u)\b\w+\b"), 'ent_dep'),
    ('head', CountVectorizer(ngram_range=(1, 1), analyzer='word', token_pattern=r"(?u)\b\w+\b"), 'ent_head'),
    ('pos', CountVectorizer(ngram_range=(1, 1), analyzer='word', token_pattern=r"(?u)\b\w+\b"), 'ent_pos'),
    ('child', CountVectorizer(ngram_range=(1, 1), analyzer='word', token_pattern=r"(?u)\b\w+\b"), 'ent_children'),
    
])

# from sklearn.svm import LinearSVC
# from sklearn.svm import SVC
# clf = SVC(kernel='linear', probability=True)
# clf = SVC(kernel='rbf', probability=True)
# clf = SVC(kernel='poly', probability=True)
# clf = LinearSVC(verbose=True)

from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier(n_estimators=20, random_state=0) # use a guassian forest
clf = RandomForestClassifier(n_estimators=200, random_state=0)
# clf = RandomForestClassifier(n_estimators=500, random_state=0) # parallel all jobs

from sklearn import preprocessing
scaler = preprocessing.StandardScaler(with_mean=False)

pipeline = make_pipeline(vec, scaler, clf)

# pipeline = make_pipeline(vec, clf)

## evaluation

In [17]:
from sklearn.pipeline import FeatureUnion, Pipeline, make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import LabelBinarizer
from sklearn import preprocessing

class LabeledNormalizer(Normalizer):
    def fit(self, X, *args, **kwargs):
        try:
            self.names = X.columns
        except:
            self.names = [str(i) for i in range(X.shape[1])]
        return super().fit(X, *args, **kwargs)
         
    def get_feature_names(self):
        return self.names
    
class SupervisionFriendlyLabelBinarizer(LabelBinarizer):
    def fit_transform(self, X, y=None):
        return super(SupervisionFriendlyLabelBinarizer,self).fit_transform(X)

class MultiLabelBinarizerWrapper(MultiLabelBinarizer):
    def fit_transform(self, X, y=None):
        return super(MultiLabelBinarizerWrapper,self).fit_transform(X)
    def get_params(self, deep=True):
        return super(MultiLabelBinarizer,self).get_params(deep=True)

# enc = OneHotEncoder(handle_unknown='ignore')
# lb = preprocessing.LabelBinarizer()

vec = ColumnTransformer([
#     ('norm', LabeledNormalizer(), ['id']),
    ('kw', TfidfVectorizer(ngram_range=(1, 1), min_df=2, token_pattern=r"(?u)\b\w+\b",), 'keyword'),
    ('loc', TfidfVectorizer(ngram_range=(1, 1), min_df=2, token_pattern=r"(?u)\b\w+\b"), 'location'),
    ('text', TfidfVectorizer(ngram_range=(1, 3), min_df=2, token_pattern=r"(?u)\b\w+\b"), 'text'),
    ('ent', CountVectorizer(ngram_range=(1, 1), analyzer='word', token_pattern=r"(?u)\b\w+\b"), 'entities'),
    ('label', CountVectorizer(ngram_range=(1, 1), analyzer='word', token_pattern=r"(?u)\b\w+\b"), 'labels'),
    ('dep', CountVectorizer(ngram_range=(1, 1), analyzer='word', token_pattern=r"(?u)\b\w+\b"), 'ent_dep'),
    ('head', CountVectorizer(ngram_range=(1, 1), analyzer='word', token_pattern=r"(?u)\b\w+\b"), 'ent_head'),
    ('pos', CountVectorizer(ngram_range=(1, 1), analyzer='word', token_pattern=r"(?u)\b\w+\b"), 'ent_pos'),
    ('child', CountVectorizer(ngram_range=(1, 1), analyzer='word', token_pattern=r"(?u)\b\w+\b"), 'ent_children'),
#     ('hash', MultiLabelBinarizerWrapper(), 'hashtag'),
#     ('hashl1', SupervisionFriendlyLabelBinarizer(), 'hashtags_have_l1_synonyms'),
#     ('hashl2', SupervisionFriendlyLabelBinarizer(), 'hashtags_have_l2_synonyms'),
#     ('subj', MultiLabelBinarizerWrapper(), 'subj'),
#     ('obj', MultiLabelBinarizerWrapper(), 'verb'),
#     ('verb', MultiLabelBinarizerWrapper(), 'obj'),
#     ('syn1', SupervisionFriendlyLabelBinarizer(), 'contains_l1_synonyms'),
#     ('syn2', SupervisionFriendlyLabelBinarizer(), 'contains_l2_synonyms'),
#     ('damage', SupervisionFriendlyLabelBinarizer(), 'contains_damaged_words'),
#     ('news', SupervisionFriendlyLabelBinarizer(), 'mentioned_news_org'),
#     ('relief', SupervisionFriendlyLabelBinarizer(), 'mentioned_relief_org'),
#     ('mentions', MultiLabelBinarizerWrapper(), 'mentions'),
#     ('orgs', MultiLabelBinarizerWrapper, 'orgs'),
#     ('gpes', MultiLabelBinarizerWrapper, 'gpes'),
#     ('facs', MultiLabelBinarizerWrapper, 'facs'),
    
])

# from sklearn.preprocessing import PolynomialFeatures
# poly = PolynomialFeatures(2)

# from sklearn.svm import LinearSVC
# from sklearn.svm import SVC
# clf = SVC(kernel='linear', probability=True)
# clf = SVC(kernel='rbf', probability=True)

# from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier(n_estimators=20, random_state=0) # use a guassian forest
# clf = RandomForestClassifier(n_estimators=500, random_state=0) # set limit to prevent overfitting
# clf = RandomForestClassifier(n_estimators=200, max_features=1000, random_state=0) # set limit to prevent overfitting

scaler = preprocessing.StandardScaler(with_mean=False)

# from sklearn.neural_network import MLPClassifier
# clf = MLPClassifier(solver='lbfgs', alpha=1e-5, activation='relu', hidden_layer_sizes=(500, 500, 500), max_iter=2000, random_state=1)

from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=500, random_state=0)

pipeline = make_pipeline(vec, scaler, clf)
# pipeline = make_pipeline(vec, clf)


# Boosting

In [18]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, cross_val_predict

def evaluate(_clf, X, y):
    report = classification_report(
        y_true=y, y_pred=cross_val_predict(pipeline, X, y, cv=5)
    )
    print(report)
    scores = cross_val_score(_clf, X, y, scoring='accuracy', cv=5)
    print('Cross-validation MSE: {:.3f} ± {:.3f}'.format(np.mean(scores), 2 * np.std(scores)))
    
    _clf.fit(X,y)
    print('Training Set Accuracy: {:.3f}'.format(_clf.score(X,y)))


import time
start_time = time.time()

evaluate(pipeline, X, y)

seconds = time.time() - start_time
print('\nEvaluation Time Taken: ', time.strftime("%H:%M:%S",time.gmtime(seconds)))

import time
start_time = time.time()

pipeline.fit(X,y)

seconds = time.time() - start_time
print('Training Time Taken:  ', time.strftime("%H:%M:%S",time.gmtime(seconds)))

              precision    recall  f1-score   support

         0.0       0.66      0.70      0.68      4305
         1.0       0.56      0.52      0.54      3198

    accuracy                           0.62      7503
   macro avg       0.61      0.61      0.61      7503
weighted avg       0.62      0.62      0.62      7503

Cross-validation MSE: 0.620 ± 0.072
Training Set Accuracy: 0.894

Evaluation Time Taken:  00:03:45
Training Time Taken:   00:00:26


## AdaBoost
### 21 features 100 ensemble
```
              precision    recall  f1-score   support

         0.0       0.65      0.69      0.67      4305
         1.0       0.54      0.49      0.51      3198

    accuracy                           0.61      7503
   macro avg       0.59      0.59      0.59      7503
weighted avg       0.60      0.61      0.60      7503

Cross-validation MSE: 0.606 ± 0.078
Training Set Accuracy: 0.898

Evaluation Time Taken:  00:03:08
Training Time Taken:   00:00:18
```

### 21 features 100 ensemble
```
              precision    recall  f1-score   support

         0.0       0.66      0.79      0.72      4305
         1.0       0.63      0.46      0.53      3198

    accuracy                           0.65      7503
   macro avg       0.65      0.63      0.63      7503
weighted avg       0.65      0.65      0.64      7503

Cross-validation MSE: 0.653 ± 0.083
Training Set Accuracy: 0.798

Evaluation Time Taken:  00:00:43
Training Time Taken:   00:00:04
```

## Adaboost with entity features (100 ensemble)
```
              precision    recall  f1-score   support

         0.0       0.66      0.79      0.72      4305
         1.0       0.61      0.44      0.51      3198

    accuracy                           0.64      7503
   macro avg       0.63      0.62      0.61      7503
weighted avg       0.64      0.64      0.63      7503

Cross-validation MSE: 0.642 ± 0.083
Training Set Accuracy: 0.801

Evaluation Time Taken:  00:00:33
Training Time Taken:   00:00:03
```

## MLP with 21 features

### Params: hidden_layer_sizes=(500, 500, 500), max_iter=2000
```
              precision    recall  f1-score   support

         0.0       0.72      0.69      0.70      4305
         1.0       0.60      0.64      0.62      3198

    accuracy                           0.67      7503
   macro avg       0.66      0.66      0.66      7503
weighted avg       0.67      0.67      0.67      7503

Cross-validation MSE: 0.665 ± 0.059
Training Set Accuracy: 0.998

Evaluation Time Taken:  09:23:30
Training Time Taken:   00:19:10
```

### Params: `solver='lbfgs', alpha=1e-5, activation='tanh', hidden_layer_sizes=(100, 100, 100), max_iter=1000`
```
              precision    recall  f1-score   support

         0.0       0.72      0.64      0.68      4305
         1.0       0.58      0.66      0.62      3198

    accuracy                           0.65      7503
   macro avg       0.65      0.65      0.65      7503
weighted avg       0.66      0.65      0.65      7503

Cross-validation MSE: 0.649 ± 0.090
Training Set Accuracy: 0.998

Evaluation Time Taken:  00:47:30
Training Time Taken:   00:05:01
```

In [12]:
from sklearn.externals import joblib

# pipeline.fit(X, y)

mlp_mdl = '../models/MLP-21-features.pkl'
joblib.dump(pipeline, mlp_mdl)
!ls -lSh $mlp_mdl

-rw-r--r-- 1 hongyu hongyu 154M Mar 29 10:09 ../models/MLP-21-features.pkl
